In [64]:
import torch 
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import pickle
from model import ATT_CNN_GRU_MultiFeature
import os
import numpy as np

# Hyperparameters
batch_size = 32
learning_rate = 0.0005
num_epochs = 200
in_channels = 3  # Number of features in the dataset
num_classes = 16  # Total number of fault classes
samples_per_class = 200  # Limit the number of samples per class

# Custom Dataset to handle pre-segmented data
class PreSegmentedDataset(Dataset):
    def __init__(self, classwise_data):
        self.data = []
        self.labels = []

        # Find the minimum number of samples across classes that have data
        min_class_samples = min(len(samples) for samples in classwise_data.values() if len(samples) > 0)
        print(f"Minimum samples per class (ignoring empty classes): {min_class_samples}")

        # Collect data and labels for each class, ensuring balanced sampling across all classes
        for i in range(min_class_samples):
            for class_label in range(num_classes):
                if classwise_data[class_label]:  # Only process if data exists for the class
                    if i < len(classwise_data[class_label]):
                        self.data.append(classwise_data[class_label][i])
                        self.labels.append(class_label)

        # If no data was collected, raise an error
        if len(self.data) == 0:
            print("Error: No data collected for any class!")
            raise RuntimeError("No data to stack, check dataset processing.")

        # Try to stack the data
        try:
            self.data = torch.stack(self.data)
            self.labels = torch.tensor(self.labels)
        except RuntimeError as e:
            print(f"Error when stacking data: {e}")
            print(f"Data list length: {len(self.data)}")
            raise e

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

# Function to create classwise data from the full dataset and limit it to `samples_per_class`
def create_limited_classwise_data(X, y, num_classes, samples_per_class):
    classwise_data = {i: [] for i in range(num_classes)}
    class_counts = {i: 0 for i in range(num_classes)}  # Count how many samples we have for each class

    for i in range(len(X)):
        class_label = int(y[i].argmax())  # Get the class label from one-hot encoded label
        if class_counts[class_label] < samples_per_class:
            classwise_data[class_label].append(X[i])
            class_counts[class_label] += 1
        
    # Print how many samples we collected for each class
    for class_label, count in class_counts.items():
        print(f"Class {class_label}: Collected {count} samples")

    if any(count == 0 for count in class_counts.values()):
        print("Error: One or more classes have zero samples, check the dataset!")

    return classwise_data

# Load the preprocessed data
with open('preprocessed_data.pkl', 'rb') as f:
    data_dict = pickle.load(f)

# Verify that each class exists in the preprocessed data
y_train = np.argmax(data_dict['y_train'], axis=1)
y_val = np.argmax(data_dict['y_val'], axis=1)
y_test = np.argmax(data_dict['y_test'], axis=1)

print("Checking class distribution in training data:")
unique_train, counts_train = np.unique(y_train, return_counts=True)
print(dict(zip(unique_train, counts_train)))

print("Checking class distribution in validation data:")
unique_val, counts_val = np.unique(y_val, return_counts=True)
print(dict(zip(unique_val, counts_val)))

print("Checking class distribution in testing data:")
unique_test, counts_test = np.unique(y_test, return_counts=True)  # Corrected this line
print(dict(zip(unique_test, counts_test)))

# Use limited samples for training, validation, and testing (20,000 samples per class)
X_train = torch.tensor(data_dict['X_train'], dtype=torch.float32)
X_val = torch.tensor(data_dict['X_val'], dtype=torch.float32)
X_test = torch.tensor(data_dict['X_test'], dtype=torch.float32)
y_train = torch.tensor(data_dict['y_train'], dtype=torch.float32)
y_val = torch.tensor(data_dict['y_val'], dtype=torch.float32)
y_test = torch.tensor(data_dict['y_test'], dtype=torch.float32)

# Create limited classwise data for training, validation, and testing
train_classwise_data = create_limited_classwise_data(X_train, y_train, num_classes, samples_per_class)
val_classwise_data = create_limited_classwise_data(X_val, y_val, num_classes, samples_per_class)
test_classwise_data = create_limited_classwise_data(X_test, y_test, num_classes, samples_per_class)

# Create datasets without segmenting again
train_dataset = PreSegmentedDataset(train_classwise_data)
val_dataset = PreSegmentedDataset(val_classwise_data)
test_dataset = PreSegmentedDataset(test_classwise_data)

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

# Initialize model
model = ATT_CNN_GRU_MultiFeature(in_channels, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Variables to track the best model
best_val_accuracy = 0.0
best_model_path = ""
best_epoch = 0  # To store the epoch when the best model is saved

# Create a directory to save models if it doesn't exist
if not os.path.exists("models"):
    os.makedirs("models")

# Training loop
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    for inputs, labels in train_loader:
        # Zero the parameter gradients
        optimizer.zero_grad()

        # Permute the input to the shape [batch_size, in_channels, sequence_length]
        inputs = inputs.permute(0, 2, 1)  # Shape should now be [batch_size, in_channels, seq_len]
        
        # Forward pass
        outputs = model(inputs)
        
        # Calculate loss
        loss = criterion(outputs, labels)  # Assuming labels are not one-hot encoded
        
        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        # Accumulate training metrics
        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

    train_accuracy = 100 * correct / total
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}, Accuracy: {train_accuracy:.2f}%')

    # Validation loop
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            # Permute the input to the shape [batch_size, in_channels, sequence_length]
            inputs = inputs.permute(0, 2, 1)
            
            # Forward pass
            outputs = model(inputs)
            
            # Calculate loss
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
    
    val_accuracy = 100 * correct / total
    print(f'Validation Loss: {val_loss/len(val_loader):.4f}, Validation Accuracy: {val_accuracy:.2f}%')

    # Save the model every 10 epochs
    if (epoch + 1) % 10 == 0:
        model_path = f"models/model_epoch_{epoch+1}_lr_{learning_rate}_bs_{batch_size}.pth"
        torch.save(model.state_dict(), model_path)
        print(f"Saved model at epoch {epoch+1} to {model_path}")

    # Save the best model based on validation accuracy
    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        best_epoch = epoch + 1  # Record the best epoch
        best_model_path = f"models/best_model_lr_{learning_rate}_bs_{batch_size}.pth"
        torch.save(model.state_dict(), best_model_path)
        print(f"New best model saved with validation accuracy: {val_accuracy:.2f}% at epoch {best_epoch}")

# Load the best model for testing
model.load_state_dict(torch.load(best_model_path))

# Test loop (testing the best model)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

model.eval()
correct = 0
total = 0
all_preds = []
all_labels = []
with torch.no_grad():
    for inputs, labels in test_loader:
        # Permute the input to the shape [batch_size, in_channels, sequence_length]
        inputs = inputs.permute(0, 2, 1)
        
        # Forward pass
        outputs = model(inputs)
        
        # Calculate the accuracy
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)
        
        # Store original and predicted labels
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

test_accuracy = 100 * correct / total
print(f'Test Accuracy: {test_accuracy:.2f}%')

# Print original and predicted class labels for each sample
print("\nOriginal vs Predicted labels:")
for original, pred in zip(all_labels, all_preds):
    print(f"Original: {original}, Predicted: {pred}")

# Print final best model path and epoch
print(f"Best model saved at: {best_model_path} with validation accuracy: {best_val_accuracy:.2f}% (epoch {best_epoch})")


Checking class distribution in training data:
{0: 3658, 1: 3629, 2: 3559, 3: 3558, 4: 7247, 5: 3750, 6: 3651, 7: 3624, 8: 3628, 9: 3586, 10: 3594, 11: 3569, 12: 3662}
Checking class distribution in validation data:
{0: 884, 1: 910, 2: 941, 3: 916, 4: 1791, 5: 951, 6: 903, 7: 888, 8: 883, 9: 879, 10: 917, 11: 922, 12: 894}
Checking class distribution in testing data:
{0: 482, 1: 471, 2: 506, 3: 532, 4: 962, 5: 577, 6: 508, 7: 506, 8: 491, 9: 541, 10: 493, 11: 513, 12: 462}
Class 0: Collected 200 samples
Class 1: Collected 200 samples
Class 2: Collected 200 samples
Class 3: Collected 200 samples
Class 4: Collected 200 samples
Class 5: Collected 200 samples
Class 6: Collected 200 samples
Class 7: Collected 200 samples
Class 8: Collected 200 samples
Class 9: Collected 200 samples
Class 10: Collected 200 samples
Class 11: Collected 200 samples
Class 12: Collected 200 samples
Class 13: Collected 0 samples
Class 14: Collected 0 samples
Class 15: Collected 0 samples
Error: One or more classes 

Shape after flattening: torch.Size([32, 8192])
Shape after flattening: torch.Size([32, 8192])
Shape after flattening: torch.Size([32, 8192])
Shape after flattening: torch.Size([32, 8192])
Shape after flattening: torch.Size([32, 8192])
Shape after flattening: torch.Size([32, 8192])
Shape after flattening: torch.Size([32, 8192])
Shape after flattening: torch.Size([32, 8192])
Shape after flattening: torch.Size([32, 8192])
Shape after flattening: torch.Size([32, 8192])
Shape after flattening: torch.Size([32, 8192])
Shape after flattening: torch.Size([32, 8192])
Shape after flattening: torch.Size([32, 8192])
Shape after flattening: torch.Size([32, 8192])
Shape after flattening: torch.Size([32, 8192])
Shape after flattening: torch.Size([32, 8192])
Shape after flattening: torch.Size([32, 8192])
Shape after flattening: torch.Size([32, 8192])
Shape after flattening: torch.Size([32, 8192])
Shape after flattening: torch.Size([32, 8192])
Shape after flattening: torch.Size([32, 8192])
Shape after f

Shape after flattening: torch.Size([32, 8192])
Shape after flattening: torch.Size([32, 8192])
Shape after flattening: torch.Size([32, 8192])
Shape after flattening: torch.Size([32, 8192])
Shape after flattening: torch.Size([32, 8192])
Shape after flattening: torch.Size([32, 8192])
Shape after flattening: torch.Size([32, 8192])
Shape after flattening: torch.Size([32, 8192])
Shape after flattening: torch.Size([32, 8192])
Shape after flattening: torch.Size([32, 8192])
Shape after flattening: torch.Size([32, 8192])
Shape after flattening: torch.Size([32, 8192])
Shape after flattening: torch.Size([32, 8192])
Shape after flattening: torch.Size([32, 8192])
Shape after flattening: torch.Size([32, 8192])
Shape after flattening: torch.Size([32, 8192])
Shape after flattening: torch.Size([32, 8192])
Shape after flattening: torch.Size([32, 8192])
Shape after flattening: torch.Size([32, 8192])
Shape after flattening: torch.Size([32, 8192])
Shape after flattening: torch.Size([32, 8192])
Shape after f

Shape after flattening: torch.Size([32, 8192])
Shape after flattening: torch.Size([32, 8192])
Shape after flattening: torch.Size([32, 8192])
Shape after flattening: torch.Size([32, 8192])
Shape after flattening: torch.Size([32, 8192])
Shape after flattening: torch.Size([32, 8192])
Shape after flattening: torch.Size([32, 8192])
Shape after flattening: torch.Size([32, 8192])
Shape after flattening: torch.Size([32, 8192])
Shape after flattening: torch.Size([32, 8192])
Shape after flattening: torch.Size([32, 8192])
Shape after flattening: torch.Size([32, 8192])
Shape after flattening: torch.Size([32, 8192])
Shape after flattening: torch.Size([32, 8192])
Shape after flattening: torch.Size([8, 8192])
Validation Loss: 1.0171, Validation Accuracy: 50.69%
New best model saved with validation accuracy: 50.69% at epoch 3
Shape after flattening: torch.Size([32, 8192])
Shape after flattening: torch.Size([32, 8192])
Shape after flattening: torch.Size([32, 8192])
Shape after flattening: torch.Size([3

Shape after flattening: torch.Size([32, 8192])
Shape after flattening: torch.Size([32, 8192])
Shape after flattening: torch.Size([32, 8192])
Shape after flattening: torch.Size([8, 8192])
Validation Loss: 1.2287, Validation Accuracy: 46.50%
Shape after flattening: torch.Size([32, 8192])
Shape after flattening: torch.Size([32, 8192])
Shape after flattening: torch.Size([32, 8192])
Shape after flattening: torch.Size([32, 8192])
Shape after flattening: torch.Size([32, 8192])
Shape after flattening: torch.Size([32, 8192])
Shape after flattening: torch.Size([32, 8192])
Shape after flattening: torch.Size([32, 8192])
Shape after flattening: torch.Size([32, 8192])
Shape after flattening: torch.Size([32, 8192])
Shape after flattening: torch.Size([32, 8192])
Shape after flattening: torch.Size([32, 8192])
Shape after flattening: torch.Size([32, 8192])
Shape after flattening: torch.Size([32, 8192])
Shape after flattening: torch.Size([32, 8192])
Shape after flattening: torch.Size([32, 8192])
Shape af

...

Shape after flattening: torch.Size([32, 8192])
Shape after flattening: torch.Size([32, 8192])
Shape after flattening: torch.Size([32, 8192])
Shape after flattening: torch.Size([32, 8192])
Shape after flattening: torch.Size([32, 8192])
Shape after flattening: torch.Size([32, 8192])
Shape after flattening: torch.Size([32, 8192])
Shape after flattening: torch.Size([32, 8192])
Shape after flattening: torch.Size([32, 8192])
Shape after flattening: torch.Size([32, 8192])
Shape after flattening: torch.Size([32, 8192])
Shape after flattening: torch.Size([32, 8192])
Shape after flattening: torch.Size([32, 8192])
Shape after flattening: torch.Size([32, 8192])
Shape after flattening: torch.Size([32, 8192])
Shape after flattening: torch.Size([32, 8192])
Shape after flattening: torch.Size([32, 8192])
Shape after flattening: torch.Size([32, 8192])
Shape after flattening: torch.Size([32, 8192])
Shape after flattening: torch.Size([32, 8192])
Shape after flattening: torch.Size([32, 8192])
Shape after f

Shape after flattening: torch.Size([32, 8192])
Shape after flattening: torch.Size([32, 8192])
Shape after flattening: torch.Size([32, 8192])
Shape after flattening: torch.Size([32, 8192])
Shape after flattening: torch.Size([32, 8192])
Shape after flattening: torch.Size([32, 8192])
Shape after flattening: torch.Size([32, 8192])
Shape after flattening: torch.Size([32, 8192])
Shape after flattening: torch.Size([32, 8192])
Shape after flattening: torch.Size([32, 8192])
Shape after flattening: torch.Size([32, 8192])
Shape after flattening: torch.Size([32, 8192])
Shape after flattening: torch.Size([32, 8192])
Shape after flattening: torch.Size([32, 8192])
Shape after flattening: torch.Size([32, 8192])
Shape after flattening: torch.Size([32, 8192])
Shape after flattening: torch.Size([32, 8192])
Shape after flattening: torch.Size([32, 8192])
Shape after flattening: torch.Size([32, 8192])
Shape after flattening: torch.Size([32, 8192])
Shape after flattening: torch.Size([32, 8192])
Shape after f

C:\Users\SIDDHARTH SINGH\AppData\Local\Temp\ipykernel_14764\3469010262.py:205: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(best_model_path

Shape after flattening: torch.Size([32, 8192])
Shape after flattening: torch.Size([32, 8192])
Shape after flattening: torch.Size([32, 8192])
Shape after flattening: torch.Size([32, 8192])
Shape after flattening: torch.Size([32, 8192])
Shape after flattening: torch.Size([32, 8192])
Shape after flattening: torch.Size([32, 8192])
Shape after flattening: torch.Size([32, 8192])
Shape after flattening: torch.Size([32, 8192])
Shape after flattening: torch.Size([32, 8192])
Shape after flattening: torch.Size([32, 8192])
Shape after flattening: torch.Size([32, 8192])
Shape after flattening: torch.Size([32, 8192])
Shape after flattening: torch.Size([32, 8192])
Shape after flattening: torch.Size([32, 8192])
Shape after flattening: torch.Size([32, 8192])
Shape after flattening: torch.Size([32, 8192])
Shape after flattening: torch.Size([32, 8192])
Shape after flattening: torch.Size([32, 8192])
Shape after flattening: torch.Size([32, 8192])
Shape after flattening: torch.Size([32, 8192])
Shape after f

Original: 7, Predicted: 7
Original: 8, Predicted: 8
Original: 9, Predicted: 9
Original: 10, Predicted: 10
Original: 11, Predicted: 11
Original: 12, Predicted: 12
Original: 0, Predicted: 9
Original: 1, Predicted: 1
Original: 2, Predicted: 2
Original: 3, Predicted: 3
Original: 4, Predicted: 4
Original: 5, Predicted: 5
Original: 6, Predicted: 6
Original: 7, Predicted: 7
Original: 8, Predicted: 8
Original: 9, Predicted: 9
Original: 10, Predicted: 10
Original: 11, Predicted: 11
Original: 12, Predicted: 12
Original: 0, Predicted: 9
Original: 1, Predicted: 1
Original: 2, Predicted: 2
Original: 3, Predicted: 3
Original: 4, Predicted: 4
Original: 5, Predicted: 5
Original: 6, Predicted: 6
Original: 7, Predicted: 7
Original: 8, Predicted: 8
Original: 9, Predicted: 9
Original: 10, Predicted: 10
Original: 11, Predicted: 11
Original: 12, Predicted: 12
Original: 0, Predicted: 0
Original: 1, Predicted: 1
Original: 2, Predicted: 2
Original: 3, Predicted: 3
Original: 4, Predicted: 4
Original: 5, Predict